# Подготовка данных

## Считаем данные в исходном csv формате

In [1]:
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


Перезапустить kernel после установки!

In [2]:
import findspark
findspark.init()

In [2]:
!hdfs dfs -ls

Found 1 items
drwxr-xr-x   - ubuntu hadoop          0 2024-10-30 07:25 data


In [3]:
!hdfs dfs -ls data

Found 1 items
-rw-r--r--   1 ubuntu hadoop 5846760913 2024-10-30 07:25 data/train.csv


Создадим SparkSession

In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("OTUS")
        .getOrCreate()
)

Загрузим данные

In [6]:
df = spark.read.csv("data/train.csv", inferSchema=True, header=True)

Поделим данные на 10 разделов и сохраним в формате parquete

In [7]:
%%time
df.count()

CPU times: user 4.75 ms, sys: 606 µs, total: 5.36 ms
Wall time: 21.1 s


101230332

In [8]:
(
    df
        .repartition(10)
        .write
        .mode("overwrite")
        .parquet("data/train.parquet")
)

Проверим размер сохраненного файла

In [9]:
!hdfs dfs -ls -h data/train.parquet

Found 11 items
-rw-r--r--   1 ubuntu hadoop          0 2024-10-30 11:38 data/train.parquet/_SUCCESS
-rw-r--r--   1 ubuntu hadoop    181.2 M 2024-10-30 11:38 data/train.parquet/part-00000-c47d3bf3-328d-4d9f-bb55-22c503696cb9-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.2 M 2024-10-30 11:38 data/train.parquet/part-00001-c47d3bf3-328d-4d9f-bb55-22c503696cb9-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.0 M 2024-10-30 11:37 data/train.parquet/part-00002-c47d3bf3-328d-4d9f-bb55-22c503696cb9-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.2 M 2024-10-30 11:38 data/train.parquet/part-00003-c47d3bf3-328d-4d9f-bb55-22c503696cb9-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.2 M 2024-10-30 11:38 data/train.parquet/part-00004-c47d3bf3-328d-4d9f-bb55-22c503696cb9-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.1 M 2024-10-30 11:37 data/train.parquet/part-00005-c47d3bf3-328d-4d9f-bb55-22c503696cb9-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.

Сравним скорость посчета строк из parquet

In [10]:
df_from_parquet = spark.read.parquet("data/train.parquet")

In [11]:
%%time
df_from_parquet.count()

CPU times: user 2.15 ms, sys: 287 µs, total: 2.44 ms
Wall time: 2 s


101230332